In [1]:
# Code to get landmarks from mediapipe and then create a bounding box around Fingers to segment them 
# for further analysis (thickness of fingers)

# Use distance transform to further correct the landmarks from mediapipe

# See if you can get tip indices from drawing a convex hull around hand??

# NOTE: Current code works on already segmented masks - this preprocessing step is required!

In [2]:
# import required libraries

import cv2 as cv # opencv
import numpy as np # for numerical calculations 
import synapseclient # synapse login etc
import synapseutils # to download files using folder structure
import pandas as pd # data frames
from matplotlib import pyplot as plt # plotting
import mediapipe as mp # for detecting hand landmarks
import seaborn as sns # for violin plot
import os # for listing files in a folder
import timeit # to track program running time

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
# login into Synapse
syn = synapseclient.login()

## hand landmark detection using mediapipe
hands = mp_hands.Hands(static_image_mode=True,
                       max_num_hands=1, # use one hand at a time 
                       min_detection_confidence=0.5)


UPGRADE AVAILABLE

A more recent version of the Synapse Client (2.5.0) is available. Your version (2.4.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 2.5.0 release notes

https://python-docs.synapse.org/build/html/news.html



Welcome, Aryton Tediarjo!



In [4]:
def getTIPLandmarks(multi_hand_landmarks, img_shape):
    # input is results.multi_hand_landmarks
    # will focus on all fingers except thumb - i.e index, middle, ring and pinky
    for hand_landmarks in multi_hand_landmarks:
        index_finger_tip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y* img_shape[0])]
        middle_finger_tip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y* img_shape[0])]
        ring_finger_tip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y* img_shape[0])]
        pinky_tip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y* img_shape[0])]

    tips_dict = {'index': index_finger_tip, 'middle': middle_finger_tip, 'ring': ring_finger_tip, 'pinky': pinky_tip}
    return(tips_dict)            

def getDIPLandmarks(multi_hand_landmarks, img_shape):
    # input is results.multi_hand_landmarks
    # will focus on all fingers except thumb - i.e index, middle, ring and pinky
    for hand_landmarks in multi_hand_landmarks:
        index_finger_dip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y* img_shape[0])]
        middle_finger_dip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y* img_shape[0])]
        ring_finger_dip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y* img_shape[0])]
        pinky_dip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y* img_shape[0])]

    dips_dict = {'index': index_finger_dip, 'middle': middle_finger_dip, 'ring': ring_finger_dip, 'pinky': pinky_dip}
    return(dips_dict) 

def getPIPLandmarks(multi_hand_landmarks, img_shape):
    # input is results.multi_hand_landmarks
    # will focus on all fingers except thumb - i.e index, middle, ring and pinky
    for hand_landmarks in multi_hand_landmarks:
        index_finger_pip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y* img_shape[0])]
        middle_finger_pip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y* img_shape[0])]
        ring_finger_pip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y* img_shape[0])]
        pinky_pip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y* img_shape[0])]

    dips_dict = {'index': index_finger_pip, 'middle': middle_finger_pip, 'ring': ring_finger_pip, 'pinky': pinky_pip}
    return(dips_dict) 

def getMCPLandmarks(multi_hand_landmarks, img_shape):
    # input is results.multi_hand_landmarks
    # will focus on all fingers except thumb - i.e index, middle, ring and pinky
    for hand_landmarks in multi_hand_landmarks:
        index_finger_mcp = [round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y* img_shape[0])]
        middle_finger_mcp = [round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y* img_shape[0])]
        ring_finger_mcp = [round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP].y* img_shape[0])]
        pinky_mcp = [round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].y* img_shape[0])]

    mcps_dict = {'index': index_finger_mcp, 'middle': middle_finger_mcp, 'ring': ring_finger_mcp, 'pinky': pinky_mcp}
    return(mcps_dict) 
    

def getTipsFromHull(tips_landmarks, hull):
    # input is TIPLandmarks and hull points
    # For each landmark we find the nearest hull point, i.e matching the hull points
    # with the tips, so that we can associate which sections of the contour belong 
    # to which finger
    
    # index finger
    index_dist = list(map(lambda x: np.linalg.norm(x-tips_landmarks['index']), hull[0]))
    index_hull_point = hull[0][index_dist.index(min(index_dist))][0]
    
    # middle finger
    middle_dist = list(map(lambda x: np.linalg.norm(x-tips_landmarks['middle']), hull[0]))
    middle_hull_point = hull[0][middle_dist.index(min(middle_dist))][0]

    # ring finger
    ring_dist = list(map(lambda x: np.linalg.norm(x-tips_landmarks['ring']), hull[0]))
    ring_hull_point = hull[0][ring_dist.index(min(ring_dist))][0]

    # pinky 
    pinky_dist = list(map(lambda x: np.linalg.norm(x-tips_landmarks['pinky']), hull[0]))
    pinky_hull_point = hull[0][pinky_dist.index(min(pinky_dist))][0]

    tips_hulls_dict = {'index': index_hull_point, 'middle': middle_hull_point, 'ring': ring_hull_point, 'pinky': pinky_hull_point}
    return(tips_hulls_dict)            

def getClosestPixelInHull(pixel_in, contour_section):
    # Given a pixel_in and contour_section, output the pixel in the contour_section that is closese to pixel_in
    index_dist = list(map(lambda x: np.linalg.norm(x-pixel_in), contour_section[0]))
    index_point = contour_section[0][index_dist.index(min(index_dist))][0]
    return(index_point)


def locatePixelInList(pixel,input_list):
    # Given a contour find the index of the input pixel (mostly one from the convex hull)
    
    temp_list = list([(input_list[0][x][0] == pixel).all() for x in range(len(input_list[0]))])
    # gives a list of true/false 
    pixel_index = temp_list.index(max(temp_list))
    # pick the true
    
    return(pixel_index)

In [5]:
def getHand(img, hand= 'left'):
    """
    Split the given two-hand image into a single hand image.
        
    :param img:   input RGB image
    :param hand:  'left' - left half of the picture 
                  'right' - right half of the image
        
    """
    rows, cols, channels = img.shape
    new_cols = round(cols/2)

    
    if hand == 'left':
        img_cropped = img[:, 0:new_cols-1,:]
    elif hand == 'right':
        img_cropped = img[:, new_cols:cols-1,:]
    else:
        print('Returning input image')
        img_cropped = img
    
    return img_cropped
    

def getBinaryImage(im, min_foreground_pixel = 10, max_foreground_pixel = 255):
    # Thresholds the given image(segmented mask with black background) to give a black-white binary image
    # with the background being white pixels and foreground(hand) being white pixels
    
    ### Convert to Gray Scale
    imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)

    ### Binary Threshold
    ret, thresh = cv.threshold(imgray, min_foreground_pixel, max_foreground_pixel, cv.THRESH_BINARY)
    
    return(thresh)

def getContoursAndHull(black_white_im):
    # Given a binary(black and white image with segmented hand mask) give output of contours, hierarchy
    # and convex hull points

    # returns only the largest contour as this is the one we want for the hand!
    
    ### Find contours
    contours, hierarchy = cv.findContours(black_white_im, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    
    ### contour lengths
    contour_lengths = [len(cc) for cc in contours]
    largest_contour_index = contour_lengths.index(max(contour_lengths))
    
    # subset contours and hierarchy to the largest contour
    contours = [contours[largest_contour_index]]
    hierarchy = np.array([[hierarchy[0][largest_contour_index]]])
    
    ### Convex Hull
    # create hull array for convex hull points
    hull = []
    # calculate points for each contour
    for i in range(len(contours)):
        # creating convex hull object for each contour
        hull.append(cv.convexHull(contours[i], False))
    
    return({'contours':contours, 'hierarchy':hierarchy, 'hull': hull})

def drawConvexHull(black_white_im, contours, hierarchy, hull, draw_Contour = True):
    # given a black white image, contours and hull, output a image with contours and hull drawn
    
    # create an empty black image
#     drawing = np.zeros((black_white_im.shape[0], black_white_im.shape[1], 3), np.uint8)
    
    drawing = black_white_im

    # draw contours and hull points
    color_contours = (0, 255, 0) # green - color for contours
    color = (255, 0, 0) # blue - color for convex hull
    
    # draw ith contour
    if draw_Contour:
        cv.drawContours(drawing, contours, -1, color_contours, 1, 8, hierarchy)
    
    # draw ith convex hull object
    cv.drawContours(drawing, hull, -1, color, 1, 8)
    
    return(drawing)



In [6]:
def cropImageFromContour(img, cnt):
    # Crop the image(img) based on the input of a closed contour(cnt), set of points
    # adapted from https://www.life2coding.com/cropping-polygon-or-non-rectangular-region-from-image-using-opencv-python/
    mask = np.zeros(img.shape[0:2], dtype=np.uint8)
    
    # draw the contours on the mask
    cv.drawContours(mask, [cnt], -1, (255, 255, 255), -1, cv.LINE_AA)
    
    res = cv.bitwise_and(img,img,mask = mask)
    rect = cv.boundingRect(cnt) # returns (x,y,w,h) of the rect
    cropped = res[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]
    
#     ## To get white background cropped image
#     ## create the white background of the same size of original image
#     wbg = np.ones_like(img, np.uint8)*255
#     cv.bitwise_not(wbg,wbg, mask=mask)
    
#     # overlap the resulted cropped image on the white background
#     dst = wbg+res
#     dst_cropped = dst[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]

    return(cropped)

def cropWarpedRect(img, rect):
    #### BEST
    
    # from https://stackoverflow.com/questions/11627362/how-to-straighten-a-rotated-rectangle-area-of-an-image-using-opencv-in-python/48553593#48553593
    # Get center, size, and angle from rect
    center, size, theta = rect
    
    box = cv.boxPoints(rect)
    box = np.int0(box)
    width = int(rect[1][0])
    height = int(rect[1][1])

    src_pts = box.astype("float32")
    dst_pts = np.array([[0, height-1],
                        [0, 0],
                        [width-1, 0],
                        [width-1, height-1]], dtype="float32")
    M = cv.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv.warpPerspective(img, M, (width, height))
    if (theta > 45):
        warped = cv.rotate(warped, cv.ROTATE_90_CLOCKWISE)
    
    return({'M':M, 'warped':warped, 'theta': theta})

def getTransformedCoords(initial_point, M, theta):
    # get transformed coords of given point from the main image in the cropped image obtained using
    # cropWarpedRect
    
    # initial point is of the form [x,y]
    # M is the perspective transform matrix from cropWarpedRect
    x,y = initial_point
    a = cv.transform(np.array([[[x,y,1]]]),M)
    xT,yT,zT = a[0][0]
    if theta > 45: # this is because in cropWarpedRect we rotate the image if theta > 45
        return([yT,xT])
    else:
        return([xT,yT])
    
def getFingerWidth(point_in_cropped_image, cropped_image):
    # get finger width by counting number of foreground pixels at a current height
    x,y = point_in_cropped_image
    width = sum(cropped_image[y,:])/255
    return(int(width))

In [7]:
def getContourSubsectionPercentage(contour_in, pixel_in, percent = 5):
    # get subsection of contour that is 5%(default) left and right to the input pixel
    input_pixel_location = locatePixelInList(pixel_in, contour_in)  
    nPixel_contour = len(contour_in[0])
    
    section_left = int(max(0, input_pixel_location - round(nPixel_contour*percent/100)))
    section_right = int(min(nPixel_contour-1, input_pixel_location + round(nPixel_contour*percent/100)))  
    
    print(section_left)
    print(section_right)
    
    subContour = [np.array(contour_in[0][section_left:section_right])]
    
    return(subContour)

In [8]:
def getContourSubsection(contour_in, pixel_on_contour, ref_pixel):
    # Given a pixel on the contour, split the contour into two sections left of the pixel
    # and right of the pixel.
    # Then find the closest points in both sections (left and right) to the ref_pixel, and 
    # then give subset the contour between these two points
    # The idea is when we give a contour of the hand, and a point on it corresponding to the
    # finger tip, we get two sections of the contour, to the left of the finger enf then right
    # We then find the points closest to these 
    
    input_pixel_location = locatePixelInList(pixel_on_contour, contour_in)  
    nPixel_contour = len(contour_in[0])
    
    # roll/shift the array so that input_pixel is the middle of array
    contour_rolled = [np.array(np.roll(contour_in[0],2*(round(nPixel_contour/2)-input_pixel_location)))]
    
    section_left = [np.array(contour_rolled[0][0:round(nPixel_contour/2)])]
    section_right = [np.array(contour_rolled[0][round(nPixel_contour/2):nPixel_contour])]
    
    closest_pixel_left = getClosestPixelInHull(ref_pixel, section_left)
    closest_pixel_right = getClosestPixelInHull(ref_pixel, section_right)
    
    closest_pixel_left_location = locatePixelInList(closest_pixel_left, contour_rolled)
    closest_pixel_right_location = locatePixelInList(closest_pixel_right, contour_rolled)
    
    subContour = [np.array(contour_rolled[0][(closest_pixel_left_location-1):closest_pixel_right_location])]
    subContour = [np.array(np.roll(subContour[0],-2*(round(nPixel_contour/2)-input_pixel_location)))]
    
#     subContour = [np.array([[pixel_on_contour], [closest_pixel_left], [closest_pixel_right]])]
#     return({'left': closest_pixel_left, 'right': closest_pixel_right})
    return(subContour)
    

In [15]:
## Target Synapse directory for segmented nails
nails_syn_target_folder = 'syn26063499'

def customSynapseUpload(file_path, file_name):
    # provenance
    provenance_set = synapseclient.Activity(
        name = file_name,
        used = 'syn25999658',
        executed = 'https://github.com/itismeghasyam/psorcastValidationAnalysis/blob/master/feature_extraction/finger_segmentation_thickness_mediapipe_pipeline.ipynb')

    test_entity = synapseclient.File(
        file_path,
        parent=nails_syn_target_folder)

    test_entity = syn.store(test_entity, activity = provenance_set)
    
    
# Function to create required directory
def create_directories():
    directories = [
        "testHullsFingers/left",
        "testHullsFingers/right",
        "segmentedFingers/left",
        "segmentedFingers/left_unrotated",
        "segmentedFingers/right",
        "segmentedFingers/right_unrotated"]
    dir_paths = []
    for directory in directories:
        dir_path = os.path.join(os.getcwd(), directory)
        dir_paths.append(dir_path)
        if not os.path.exists(directory):
            os.makedirs(dir_path)
        else:
            pass
    return(dir_paths)

In [16]:
create_directories()

['/Users/atediarjo/Documents/SageBionetworks/psorcast-validation-analysis/analysis/handImaging_analysis/testHullsFingers/left',
 '/Users/atediarjo/Documents/SageBionetworks/psorcast-validation-analysis/analysis/handImaging_analysis/testHullsFingers/right',
 '/Users/atediarjo/Documents/SageBionetworks/psorcast-validation-analysis/analysis/handImaging_analysis/segmentedFingers/left',
 '/Users/atediarjo/Documents/SageBionetworks/psorcast-validation-analysis/analysis/handImaging_analysis/segmentedFingers/left_unrotated',
 '/Users/atediarjo/Documents/SageBionetworks/psorcast-validation-analysis/analysis/handImaging_analysis/segmentedFingers/right',
 '/Users/atediarjo/Documents/SageBionetworks/psorcast-validation-analysis/analysis/handImaging_analysis/segmentedFingers/right_unrotated']

In [10]:
# In the working folder (..../), have the following folder structure
# ..../hand_segmentation_psorcast Jun 16 2021 08_51 Dan/
# ..../testHullsFingers
# ..../testHullsFingers/left
# ..../testHullsFingers/right
# ..../segmentedFingers
# ..../segmentedFingers/left
# ..../segmentedFingers/left_unrotated
# ..../segmentedFingers/right
# ..../segmentedFingers/right_unrotated

In [11]:
## Getting hands images from Synapse
hand_masks_synapse_id = 'syn25999658' # Folder containing all merged hand images and masks
img_all_directory = 'hand_segmentation_psorcast Jun 16 2021 08_51 Dan/' # Local path for storing images from Synapse
# Hand mask images form slack (check June 16 - Chat with Dan Webster, Aryton Tediarjo and Meghasyam Tummalacherla)
# Also in https://www.synapse.org/#!Synapse:syn25999658

# Download the curated hand image files from Synapse
hands_all_files = synapseutils.syncFromSynapse(syn = syn, entity= hand_masks_synapse_id, path= img_all_directory)

all_files = os.listdir(img_all_directory)

fuse_files = list(filter(lambda x: 'fuse' in x, all_files))
orig_files = list(map(lambda x: x[:-11], fuse_files)) # remove ___fuse.png

target_directory = 'testHullsFingers'
target_fingers_directory = 'segmentedFingers'

In [17]:
#### LEFT HAND

left_fail_index = 0
left_pass_index = 0
left_fingers_rects = {} # save image name and rect dictionaries, i.e image vs finger bounding boxes

left_finger_img_shape = {} # Image shape of the cropped finger images

left_img_shape = {} # Shape of the whole hand input image

left_tip_positions = {} # save tip landmark positions of cropped finger images
left_dip_positions = {} # save dip landmark positions of cropped finger images
left_pip_positions = {} # save pip landmark positions of cropped finger images

left_tip_widths = {} # save finger tip widths (number of foreground pixels)
left_dip_widths = {} # save finger dip widths (number of foreground pixels)
left_pip_widths = {} # save finger pip widths (number of foreground pixels)


for index in range(len(fuse_files)):
    
    # update path for current image
    current_image_path = img_all_directory + '/' + fuse_files[index]
    current_orig_path = img_all_directory + '/' + orig_files[index]
    
    print(current_image_path)
    
    # update target path for the contoured image for the current image
    current_left_target_path = target_directory + '/' + 'left/' + fuse_files[index]
        
    # update target path for segmented fingers from the current image (ROTATED)
    current_left_fingers_target_path = target_fingers_directory + '/' + 'left'

    # update target path for segmented fingers from the current image (UN-ROTATED)
    current_left_fingers_target_path_unrotated = target_fingers_directory + '/' + 'left_unrotated'

    # read images
    img = cv.imread(current_image_path)
    orig_img = cv.imread(current_orig_path)
    
    # Masks of left and right img (fuse files)
    left_img = getHand(img, 'left')
    
    # Actual photos
    left_orig_img = getHand(orig_img, 'left')
    
    # clones of actual images
    left_orig_img_clone = left_orig_img.copy()
    left_img_clone = left_img.copy()
    
    # resize image [for faster calculation, and mediapipe usually takes in small images of size
    # 300 x 300][https://github.com/google/mediapipe/blob/master/mediapipe/calculators/image/scale_image_calculator.cc]
    img_shape  = left_orig_img.shape
    resize_factor = round(300/max(img_shape),3) # resize max(length, width) to 300 pixels
    left_orig_img_resize = cv.resize(left_orig_img, None, fx = resize_factor , fy = resize_factor, interpolation = cv.INTER_AREA)
    
    color_contours = (0, 255, 0) # green - color for contours
    
    ### left hand
    bw_img = getBinaryImage(left_img)
    
    ## get distance transform image
    dist_trans_img = cv.distanceTransform(bw_img, cv.DIST_L2, 3)
    # using Eucledean distance L2, mask size 3x3. Check https://docs.opencv.org/3.4/d7/d1b/group__imgproc__misc.html#gaa2bfbebbc5c320526897996aafa1d8eb

    ###
    contours_hull = getContoursAndHull(bw_img)

    ###
    dw_img = drawConvexHull(left_orig_img, contours_hull['contours'], contours_hull['hierarchy'], contours_hull['hull'])

    # apply mediapipe to get hand landmarks
    results = hands.process(cv.cvtColor(left_orig_img_resize, cv.COLOR_BGR2RGB))

    # to draw all landmarks from mediapipe
    if not results.multi_hand_landmarks:
        cv.imwrite(current_left_target_path, dw_img)
        left_fingers_rects[orig_files[index]] = {}        
        left_fail_index = left_fail_index + 1
    else:
        dw_img = drawConvexHull(left_orig_img, contours_hull['contours'], contours_hull['hierarchy'], contours_hull['hull'], draw_Contour=True)
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(dw_img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # get tip landmarks from results
        tips_landmarks = getTIPLandmarks(results.multi_hand_landmarks, left_img.shape)

        # get dip landmarks from results
        dips_landmarks = getDIPLandmarks(results.multi_hand_landmarks, left_img.shape)

        # get pip landmarks from results
        pips_landmarks = getPIPLandmarks(results.multi_hand_landmarks, left_img.shape)
        
        # get MCP landmarks from results
        mcps_landmarks = getMCPLandmarks(results.multi_hand_landmarks, left_img.shape)
        
        # get points closest to tips from hull
        tips_hull = getTipsFromHull(tips_landmarks, contours_hull['hull'])
        
        # get subcontours for each finger
        subContours = {}
        
        # get minimum bounding rectangle(min area) for each subContour
        subContourRects = {}
        subContourBoxes = {}
        
        # get tip, pip and dip joint positions in cropped images for each finger for each record
        tipPositionInCroppedImg = {}
        dipPositionInCroppedImg = {}
        pipPositionInCroppedImg = {}
        
        # get tip, pip and dip joint thickness from cropped image by counting the number of foreground
        # pixels around the tip, pip, dip joints. [just the y position of the landmark is enough, i.e the
        # row in the image, we then count number of foreground pixels in that row(x-direction) as a proxy for
        # finger width at that point]
        tipWidthInCroppedImg = {}
        dipWidthInCroppedImg = {}
        pipWidthInCroppedImg = {}
        
        # save shape of cropped finger images
        croppedFingerShape = {}
        
        for finger_key in tips_hull:
            subContours[finger_key] = getContourSubsection(contours_hull['contours'], tips_hull[finger_key], mcps_landmarks[finger_key])
            rect = cv.minAreaRect(subContours[finger_key][0])
            box = cv.boxPoints(rect)
            box = np.int0(box)
            subContourRects[finger_key] = rect
            subContourBoxes[finger_key] = [box]
    
#         draw SubContours finger
        for finger_key in subContours:
            if (len(subContours[finger_key][0]) > 0):
                cv.drawContours(dw_img, subContours[finger_key], -1, (255,0,0), 1, 8, contours_hull['hierarchy'])
                cv.drawContours(dw_img, subContourBoxes[finger_key],-1,(0,0,255))

            # rotated via rects (crop in masked image, because we need finger thickness)
            crop_result = cropWarpedRect(left_img_clone, subContourRects[finger_key])
            cropped_finger = crop_result['warped']
            cropped_finger_bw = getBinaryImage(cropped_finger)
            perspective_transform_mat = crop_result['M']
            cropped_finger_theta = crop_result['theta']
            
            # get finger landmarks positions in cropped images
            tipPositionInCroppedImg[finger_key] = getTransformedCoords(tips_landmarks[finger_key],perspective_transform_mat, cropped_finger_theta)
            dipPositionInCroppedImg[finger_key] = getTransformedCoords(dips_landmarks[finger_key],perspective_transform_mat, cropped_finger_theta)
            pipPositionInCroppedImg[finger_key] = getTransformedCoords(pips_landmarks[finger_key],perspective_transform_mat, cropped_finger_theta)
            
            # get finger widths at landmark positions in cropped images
            tipWidthInCroppedImg[finger_key] = getFingerWidth(tipPositionInCroppedImg[finger_key], cropped_finger_bw)
            dipWidthInCroppedImg[finger_key] = getFingerWidth(dipPositionInCroppedImg[finger_key], cropped_finger_bw)
            pipWidthInCroppedImg[finger_key] = getFingerWidth(pipPositionInCroppedImg[finger_key], cropped_finger_bw)
            
            # save image shape of cropped fingers
            croppedFingerShape[finger_key] = cropped_finger_bw.shape
            
            curr_file_name = 'left_' + finger_key + '_' + fuse_files[index]            
            curr_path = current_left_fingers_target_path + '/' + curr_file_name
            if min(cropped_finger.shape) >0 :
                cv.imwrite(curr_path, cropped_finger)
                # upload this corrected cropped finger to synapse via customSynpaseUpload
                customSynapseUpload(curr_path, curr_file_name)
                
            # unrotated fingers (crop in masked image, because we need finger thickness)
            cropped_finger_unrotated = cropImageFromContour(left_img_clone, subContourBoxes[finger_key][0])
            curr_file_name = 'left_' + finger_key + '_' + fuse_files[index]
            curr_path = current_left_fingers_target_path_unrotated + '/' + curr_file_name
            if min(cropped_finger_unrotated.shape) >0 :
                cv.imwrite(curr_path, cropped_finger_unrotated)

            
        cv.imwrite(current_left_target_path, dw_img)
        left_fingers_rects[orig_files[index]] = subContourRects
        left_tip_positions[orig_files[index]] = tipPositionInCroppedImg
        left_dip_positions[orig_files[index]] = dipPositionInCroppedImg
        left_pip_positions[orig_files[index]] = pipPositionInCroppedImg
        left_tip_widths[orig_files[index]] = tipWidthInCroppedImg
        left_dip_widths[orig_files[index]] = dipWidthInCroppedImg
        left_pip_widths[orig_files[index]] = pipWidthInCroppedImg
        left_finger_img_shape[orig_files[index]] = croppedFingerShape
        left_img_shape[orig_files[index]] = [bw_img.shape]
        
        left_pass_index = left_pass_index + 1

print('left fail percent')
print(100*left_fail_index/(left_fail_index+left_pass_index))
    

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_032_handImaging_2020-12-02_20.49.05.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_008_handImaging_2020-03-04_15.43.41.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage

In [18]:
# left cropped finger image shapes into a dataframe
aa_left_finger_shapes = pd.DataFrame.from_dict(left_finger_img_shape, orient = 'index')
aa_left_finger_shapes.columns = 'left_' + aa_left_finger_shapes.columns + '_shape'
aa_left_finger_shapes['image'] = aa_left_finger_shapes.index
aa_left_finger_shapes.index = range(len(aa_left_finger_shapes.index))

# left hand image shapes into a dataframe
aa_left_hand_shapes = pd.DataFrame.from_dict(left_img_shape, orient = 'index')
aa_left_hand_shapes = aa_left_hand_shapes.rename(columns={0: "left_hand_shape"})
aa_left_hand_shapes['image'] = aa_left_hand_shapes.index
aa_left_hand_shapes.index = range(len(aa_left_hand_shapes.index))

# left_tip_positions into a dataframe
aa_left_tip =  pd.DataFrame.from_dict(left_tip_positions,orient = 'index')
aa_left_tip.columns = 'left_' + aa_left_tip.columns + '_tip' + '_location'
aa_left_tip['image'] = aa_left_tip.index
aa_left_tip.index = range(len(aa_left_tip.index))

# left_dip_positions into a dataframe
aa_left_dip =  pd.DataFrame.from_dict(left_dip_positions,orient = 'index')
aa_left_dip.columns = 'left_' + aa_left_dip.columns + '_dip' + '_location'
aa_left_dip['image'] = aa_left_dip.index
aa_left_dip.index = range(len(aa_left_dip.index))

# left_pip_positions into a dataframe
aa_left_pip =  pd.DataFrame.from_dict(left_pip_positions,orient = 'index')
aa_left_pip.columns = 'left_' + aa_left_pip.columns + '_pip' + '_location'
aa_left_pip['image'] = aa_left_pip.index
aa_left_pip.index = range(len(aa_left_pip.index))

# merge all finger landmarks for left hand
aa_landmarks = pd.merge(aa_left_tip, aa_left_dip,on='image', how='outer')
aa_landmarks = pd.merge(aa_landmarks, aa_left_pip, on = 'image', how='outer')
aa_landmarks = pd.merge(aa_landmarks, aa_left_finger_shapes, on = 'image', how='outer')
aa_landmarks = pd.merge(aa_landmarks, aa_left_hand_shapes, on = 'image', how = 'outer')

# Lengths of fingers from dip to pip (in pixels)
# Assuming the fingers are oriented straight in the cropped image
aa_landmarks['left_index_pip_to_dip_length'] = aa_landmarks['left_index_pip_location'].apply(lambda x: x[1]) - aa_landmarks['left_index_dip_location'].apply(lambda x:x[1])
aa_landmarks['left_middle_pip_to_dip_length'] = aa_landmarks['left_middle_pip_location'].apply(lambda x: x[1]) - aa_landmarks['left_middle_dip_location'].apply(lambda x:x[1])
aa_landmarks['left_ring_pip_to_dip_length'] = aa_landmarks['left_ring_pip_location'].apply(lambda x: x[1]) - aa_landmarks['left_ring_dip_location'].apply(lambda x:x[1])
aa_landmarks['left_pinky_pip_to_dip_length'] = aa_landmarks['left_pinky_pip_location'].apply(lambda x: x[1]) - aa_landmarks['left_pinky_dip_location'].apply(lambda x:x[1])

# left hand landmarks
aa_left_landmarks = aa_landmarks

In [19]:
# left_tip_widths into a dataframe
aa_left_tip =  pd.DataFrame.from_dict(left_tip_widths,orient = 'index')
aa_left_tip.columns = 'left_' + aa_left_tip.columns + '_tip' + '_width'
aa_left_tip['image'] = aa_left_tip.index
aa_left_tip.index = range(len(aa_left_tip.index))

# left_dip_widths into a dataframe
aa_left_dip =  pd.DataFrame.from_dict(left_dip_widths,orient = 'index')
aa_left_dip.columns = 'left_' + aa_left_dip.columns + '_dip' + '_width'
aa_left_dip['image'] = aa_left_dip.index
aa_left_dip.index = range(len(aa_left_dip.index))

# left_pip_widths into a dataframe
aa_left_pip =  pd.DataFrame.from_dict(left_pip_widths,orient = 'index')
aa_left_pip.columns = 'left_' + aa_left_pip.columns + '_pip' + '_width'
aa_left_pip['image'] = aa_left_pip.index
aa_left_pip.index = range(len(aa_left_pip.index))

# merge all finger landmarks for left hand
aa_widths = pd.merge(aa_left_tip, aa_left_dip,on='image', how='outer')
aa_widths = pd.merge(aa_widths, aa_left_pip, on = 'image', how='outer')
aa_widths['left_index_pip_to_dip_width'] = aa_widths['left_index_pip_width']/aa_widths['left_index_dip_width']
aa_widths['left_middle_pip_to_dip_width'] = aa_widths['left_middle_pip_width']/aa_widths['left_middle_dip_width']
aa_widths['left_ring_pip_to_dip_width'] = aa_widths['left_ring_pip_width']/aa_widths['left_ring_dip_width']
aa_widths['left_pinky_pip_to_dip_width'] = aa_widths['left_pinky_pip_width']/aa_widths['left_pinky_dip_width']

# left hand widths
aa_left_widths = aa_widths

In [20]:
# All finger, hand and landmark dimensions related to the left hand
aa_left_all = pd.merge(aa_left_landmarks, aa_left_widths, on = 'image', how = 'outer')
# aa_left_all

In [ ]:
#### RIGHT HAND

right_fail_index = 0
right_pass_index = 0
right_fingers_rects = {} # save image name and rect dictionaries, i.e image vs finger bounding boxes

right_finger_img_shape = {} # Image shape of the cropped finger images

right_img_shape = {} # Shape of the whole hand input image

right_tip_positions = {} # save tip landmark positions of cropped finger images
right_dip_positions = {} # save dip landmark positions of cropped finger images
right_pip_positions = {} # save pip landmark positions of cropped finger images

right_tip_widths = {} # save finger tip widths (number of foreground pixels)
right_dip_widths = {} # save finger dip widths (number of foreground pixels)
right_pip_widths = {} # save finger pip widths (number of foreground pixels)


for index in range(len(fuse_files)):
    
    # update path for current image
    current_image_path = img_all_directory + '/' + fuse_files[index]
    current_orig_path = img_all_directory + '/' + orig_files[index]
    
    print(current_image_path)
    
    # update target path for the contoured image for the current image
    current_right_target_path = target_directory + '/' + 'right/' + fuse_files[index]
        
    # update target path for segmented fingers from the current image (ROTATED)
    current_right_fingers_target_path = target_fingers_directory + '/' + 'right'

    # update target path for segmented fingers from the current image (UN-ROTATED)
    current_right_fingers_target_path_unrotated = target_fingers_directory + '/' + 'right_unrotated'

    # read images
    img = cv.imread(current_image_path)
    orig_img = cv.imread(current_orig_path)
    
    # Masks of right and right img (fuse files)
    right_img = getHand(img, 'right')
    
    # Actual photos
    right_orig_img = getHand(orig_img, 'right')
    
    # clones of actual images
    right_orig_img_clone = right_orig_img.copy()
    right_img_clone = right_img.copy()
    
    # resize image [for faster calculation, and mediapipe usually takes in small images of size
    # 300 x 300][https://github.com/google/mediapipe/blob/master/mediapipe/calculators/image/scale_image_calculator.cc]
    img_shape  = right_orig_img.shape
    resize_factor = round(300/max(img_shape),3) # resize max(length, width) to 300 pixels
    right_orig_img_resize = cv.resize(right_orig_img, None, fx = resize_factor , fy = resize_factor, interpolation = cv.INTER_AREA)
    
    color_contours = (0, 255, 0) # green - color for contours
    
    ### right hand
    bw_img = getBinaryImage(right_img)
    
    ## get distance transform image
    dist_trans_img = cv.distanceTransform(bw_img, cv.DIST_L2, 3)
    # using Eucledean distance L2, mask size 3x3. Check https://docs.opencv.org/3.4/d7/d1b/group__imgproc__misc.html#gaa2bfbebbc5c320526897996aafa1d8eb

    ###
    contours_hull = getContoursAndHull(bw_img)

    ###
    dw_img = drawConvexHull(right_orig_img, contours_hull['contours'], contours_hull['hierarchy'], contours_hull['hull'])

    # apply mediapipe to get hand landmarks
    results = hands.process(cv.cvtColor(right_orig_img_resize, cv.COLOR_BGR2RGB))

    # to draw all landmarks from mediapipe
    if not results.multi_hand_landmarks:
        cv.imwrite(current_right_target_path, dw_img)
        right_fingers_rects[orig_files[index]] = {}        
        right_fail_index = right_fail_index + 1
    else:
        dw_img = drawConvexHull(right_orig_img, contours_hull['contours'], contours_hull['hierarchy'], contours_hull['hull'], draw_Contour=True)
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(dw_img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # get tip landmarks from results
        tips_landmarks = getTIPLandmarks(results.multi_hand_landmarks, right_img.shape)

        # get dip landmarks from results
        dips_landmarks = getDIPLandmarks(results.multi_hand_landmarks, right_img.shape)

        # get pip landmarks from results
        pips_landmarks = getPIPLandmarks(results.multi_hand_landmarks, right_img.shape)
        
        # get MCP landmarks from results
        mcps_landmarks = getMCPLandmarks(results.multi_hand_landmarks, right_img.shape)
        
        # get points closest to tips from hull
        tips_hull = getTipsFromHull(tips_landmarks, contours_hull['hull'])
        
        # get subcontours for each finger
        subContours = {}
        
        # get minimum bounding rectangle(min area) for each subContour
        subContourRects = {}
        subContourBoxes = {}
        
        # get tip, pip and dip joint positions in cropped images for each finger for each record
        tipPositionInCroppedImg = {}
        dipPositionInCroppedImg = {}
        pipPositionInCroppedImg = {}
        
        # get tip, pip and dip joint thickness from cropped image by counting the number of foreground
        # pixels around the tip, pip, dip joints. [just the y position of the landmark is enough, i.e the
        # row in the image, we then count number of foreground pixels in that row(x-direction) as a proxy for
        # finger width at that point]
        tipWidthInCroppedImg = {}
        dipWidthInCroppedImg = {}
        pipWidthInCroppedImg = {}
        
        # save shape of cropped finger images
        croppedFingerShape = {}
        
        for finger_key in tips_hull:
            subContours[finger_key] = getContourSubsection(contours_hull['contours'], tips_hull[finger_key], mcps_landmarks[finger_key])
            rect = cv.minAreaRect(subContours[finger_key][0])
            box = cv.boxPoints(rect)
            box = np.int0(box)
            subContourRects[finger_key] = rect
            subContourBoxes[finger_key] = [box]
    
#         draw SubContours finger
        for finger_key in subContours:
            if (len(subContours[finger_key][0]) > 0):
                cv.drawContours(dw_img, subContours[finger_key], -1, (255,0,0), 1, 8, contours_hull['hierarchy'])
                cv.drawContours(dw_img, subContourBoxes[finger_key],-1,(0,0,255))

            # rotated via rects (crop in masked image, because we need finger thickness)
            crop_result = cropWarpedRect(right_img_clone, subContourRects[finger_key])
            cropped_finger = crop_result['warped']
            cropped_finger_bw = getBinaryImage(cropped_finger)
            perspective_transform_mat = crop_result['M']
            cropped_finger_theta = crop_result['theta']
            
            # get finger landmarks positions in cropped images
            tipPositionInCroppedImg[finger_key] = getTransformedCoords(tips_landmarks[finger_key],perspective_transform_mat, cropped_finger_theta)
            dipPositionInCroppedImg[finger_key] = getTransformedCoords(dips_landmarks[finger_key],perspective_transform_mat, cropped_finger_theta)
            pipPositionInCroppedImg[finger_key] = getTransformedCoords(pips_landmarks[finger_key],perspective_transform_mat, cropped_finger_theta)
            
            # get finger widths at landmark positions in cropped images
            tipWidthInCroppedImg[finger_key] = getFingerWidth(tipPositionInCroppedImg[finger_key], cropped_finger_bw)
            dipWidthInCroppedImg[finger_key] = getFingerWidth(dipPositionInCroppedImg[finger_key], cropped_finger_bw)
            pipWidthInCroppedImg[finger_key] = getFingerWidth(pipPositionInCroppedImg[finger_key], cropped_finger_bw)
            
            # save image shape of cropped fingers
            croppedFingerShape[finger_key] = cropped_finger_bw.shape
            
            curr_file_name = 'right_' + finger_key + '_' + fuse_files[index]            
            curr_path = current_right_fingers_target_path + '/' + curr_file_name
            if min(cropped_finger.shape) >0 :
                cv.imwrite(curr_path, cropped_finger)
                # upload this corrected cropped finger to synapse via customSynpaseUpload
                customSynapseUpload(curr_path, curr_file_name)
                
            # unrotated fingers (crop in masked image, because we need finger thickness)
            cropped_finger_unrotated = cropImageFromContour(right_img_clone, subContourBoxes[finger_key][0])
            curr_file_name = 'right_' + finger_key + '_' + fuse_files[index]
            curr_path = current_right_fingers_target_path_unrotated + '/' + curr_file_name
            if min(cropped_finger_unrotated.shape) >0 :
                cv.imwrite(curr_path, cropped_finger_unrotated)

            
        cv.imwrite(current_right_target_path, dw_img)
        right_fingers_rects[orig_files[index]] = subContourRects
        right_tip_positions[orig_files[index]] = tipPositionInCroppedImg
        right_dip_positions[orig_files[index]] = dipPositionInCroppedImg
        right_pip_positions[orig_files[index]] = pipPositionInCroppedImg
        right_tip_widths[orig_files[index]] = tipWidthInCroppedImg
        right_dip_widths[orig_files[index]] = dipWidthInCroppedImg
        right_pip_widths[orig_files[index]] = pipWidthInCroppedImg
        right_finger_img_shape[orig_files[index]] = croppedFingerShape
        right_img_shape[orig_files[index]] = [bw_img.shape]
        
        right_pass_index = right_pass_index + 1

print('right fail percent')
print(100*right_fail_index/(right_fail_index+right_pass_index))
    

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_032_handImaging_2020-12-02_20.49.05.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_008_handImaging_2020-03-04_15.43.41.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage

In [ ]:
# right cropped finger image shapes into a dataframe
aa_right_finger_shapes = pd.DataFrame.from_dict(right_finger_img_shape, orient = 'index')
aa_right_finger_shapes.columns = 'right_' + aa_right_finger_shapes.columns + '_shape'
aa_right_finger_shapes['image'] = aa_right_finger_shapes.index
aa_right_finger_shapes.index = range(len(aa_right_finger_shapes.index))

# right hand image shapes into a dataframe
aa_right_hand_shapes = pd.DataFrame.from_dict(right_img_shape, orient = 'index')
aa_right_hand_shapes = aa_right_hand_shapes.rename(columns={0: "right_hand_shape"})
aa_right_hand_shapes['image'] = aa_right_hand_shapes.index
aa_right_hand_shapes.index = range(len(aa_right_hand_shapes.index))

# right_tip_positions into a dataframe
aa_right_tip =  pd.DataFrame.from_dict(right_tip_positions,orient = 'index')
aa_right_tip.columns = 'right_' + aa_right_tip.columns + '_tip' + '_location'
aa_right_tip['image'] = aa_right_tip.index
aa_right_tip.index = range(len(aa_right_tip.index))

# right_dip_positions into a dataframe
aa_right_dip =  pd.DataFrame.from_dict(right_dip_positions,orient = 'index')
aa_right_dip.columns = 'right_' + aa_right_dip.columns + '_dip' + '_location'
aa_right_dip['image'] = aa_right_dip.index
aa_right_dip.index = range(len(aa_right_dip.index))

# right_pip_positions into a dataframe
aa_right_pip =  pd.DataFrame.from_dict(right_pip_positions,orient = 'index')
aa_right_pip.columns = 'right_' + aa_right_pip.columns + '_pip' + '_location'
aa_right_pip['image'] = aa_right_pip.index
aa_right_pip.index = range(len(aa_right_pip.index))

# merge all finger landmarks for right hand
aa_landmarks = pd.merge(aa_right_tip, aa_right_dip,on='image', how='outer')
aa_landmarks = pd.merge(aa_landmarks, aa_right_pip, on = 'image', how='outer')
aa_landmarks = pd.merge(aa_landmarks, aa_right_finger_shapes, on = 'image', how='outer')
aa_landmarks = pd.merge(aa_landmarks, aa_right_hand_shapes, on = 'image', how = 'outer')

# Lengths of fingers from dip to pip (in pixels)
# Assuming the fingers are oriented straight in the cropped image
aa_landmarks['right_index_pip_to_dip_length'] = aa_landmarks['right_index_pip_location'].apply(lambda x: x[1]) - aa_landmarks['right_index_dip_location'].apply(lambda x:x[1])
aa_landmarks['right_middle_pip_to_dip_length'] = aa_landmarks['right_middle_pip_location'].apply(lambda x: x[1]) - aa_landmarks['right_middle_dip_location'].apply(lambda x:x[1])
aa_landmarks['right_ring_pip_to_dip_length'] = aa_landmarks['right_ring_pip_location'].apply(lambda x: x[1]) - aa_landmarks['right_ring_dip_location'].apply(lambda x:x[1])
aa_landmarks['right_pinky_pip_to_dip_length'] = aa_landmarks['right_pinky_pip_location'].apply(lambda x: x[1]) - aa_landmarks['right_pinky_dip_location'].apply(lambda x:x[1])

# right hand landmarks
aa_right_landmarks = aa_landmarks

In [ ]:
# right_tip_widths into a dataframe
aa_right_tip =  pd.DataFrame.from_dict(right_tip_widths,orient = 'index')
aa_right_tip.columns = 'right_' + aa_right_tip.columns + '_tip' + '_width'
aa_right_tip['image'] = aa_right_tip.index
aa_right_tip.index = range(len(aa_right_tip.index))

# right_dip_widths into a dataframe
aa_right_dip =  pd.DataFrame.from_dict(right_dip_widths,orient = 'index')
aa_right_dip.columns = 'right_' + aa_right_dip.columns + '_dip' + '_width'
aa_right_dip['image'] = aa_right_dip.index
aa_right_dip.index = range(len(aa_right_dip.index))

# right_pip_widths into a dataframe
aa_right_pip =  pd.DataFrame.from_dict(right_pip_widths,orient = 'index')
aa_right_pip.columns = 'right_' + aa_right_pip.columns + '_pip' + '_width'
aa_right_pip['image'] = aa_right_pip.index
aa_right_pip.index = range(len(aa_right_pip.index))

# merge all finger landmarks for right hand
aa_widths = pd.merge(aa_right_tip, aa_right_dip,on='image', how='outer')
aa_widths = pd.merge(aa_widths, aa_right_pip, on = 'image', how='outer')
aa_widths['right_index_pip_to_dip_width'] = aa_widths['right_index_pip_width']/aa_widths['right_index_dip_width']
aa_widths['right_middle_pip_to_dip_width'] = aa_widths['right_middle_pip_width']/aa_widths['right_middle_dip_width']
aa_widths['right_ring_pip_to_dip_width'] = aa_widths['right_ring_pip_width']/aa_widths['right_ring_dip_width']
aa_widths['right_pinky_pip_to_dip_width'] = aa_widths['right_pinky_pip_width']/aa_widths['right_pinky_dip_width']

# right hand widths
aa_right_widths = aa_widths

In [ ]:
# All finger, hand and landmark dimensions related to the right hand
aa_right_all = pd.merge(aa_right_landmarks, aa_right_widths, on = 'image', how = 'outer')
# aa_right_all

In [ ]:
# Merge all finger, hand and landmark dimensions for both hands
aa_all = pd.merge(aa_right_all, aa_left_all, on = 'image', how = 'outer')

### Upload nail bounding rects to Synapse

## Write to csv
aa_all.to_csv('finger_landmarks.csv')

# Synapse target folder
syn_target_folder = 'syn22342373'

# Upload results to Synapse

# provenance
provenance_set = synapseclient.Activity(
    name='finger_landmarks.csv',
    description='Locations and widths of each finger landmarks like pip, tip, dip for both hands',
    used = 'syn25999658',
    executed = 'https://github.com/itismeghasyam/psorcastValidationAnalysis/blob/master/feature_extraction/finger_segmentation_thickness_mediapipe_pipeline.ipynb')

test_entity = synapseclient.File(
    'finger_landmarks.csv',
    description='Locations and widths of each finger landmarks like pip, tip, dip for both hands',
    parent=syn_target_folder)

test_entity = syn.store(test_entity, activity = provenance_set)

